# Union Assistant JSON Responses

This notebook combines responses from text files in the folder `response_analysis_logs` that start with `FINAL_DATA_RUN_02_`. Only the assistant responses that begin with the literal string `'''json` are extracted and combined into one large DataFrame.

In [12]:
import os
import glob
import re
import pandas as pd

# Define the directory and file pattern
log_folder = 'response_analysis_logs'
file_pattern = os.path.join(log_folder, 'FINAL_DATA_RUN_02_*')

# List all matching files
files = glob.glob(file_pattern)
print(f'Found {len(files)} files.')

# Define a regex pattern to extract blocks that start with '''json at the beginning of a line
# and continue until the next line that starts with ''' or end-of-file.
block_pattern = re.compile(r"(?m)^(?:'''json)[\s\S]*?(?=^(?:'''|$))")

responses = []

for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        content = f.read()
        # Find all blocks starting with '''json
        matches = block_pattern.findall(content)
        print(f"File: {file} -> {len(matches)} match(es)")
        responses.extend(matches)

print(f'Total responses collected: {len(responses)}')

# Create a DataFrame from the collected responses
df_responses = pd.DataFrame(responses, columns=['assistant_response'])

# Display the first few rows of the DataFrame
df_responses.head()

Found 11 files.
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_20-56-40_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_20-46-16_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_20-36-04_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_20-21-01_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_21-07-05_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_20-51-28_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_20-25-28_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_21-01-42_ai_response_analysis.json -> 0 match(es)
File: response_analysis_logs/FINAL_DATA_RUN_02_2025-02-03_20-30-42_ai_response_analysis.json -> 0 match(es)
File: respon

,assistant_response


In [13]:
import os
import glob
import re
import pandas as pd

# Define the directory and file pattern
log_folder = 'response_analysis_logs'
file_pattern = os.path.join(log_folder, 'FINAL_DATA_RUN_02_*')

# List all matching files
files = glob.glob(file_pattern)
print(f'Found {len(files)} files.')



responses = []

for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        # Each file is expected to contain a JSON list
        data = json.load(f)
        responses.extend(data)

print(f'Total responses collected: {len(responses)}')

# Create a DataFrame from the collected responses
df_responses = pd.DataFrame(responses)

# Display the first few rows of the DataFrame
# df_responses.head()
df_responses

Found 11 files.
Total responses collected: 143


,role,content
0,user,You are a skilled analyst with experience in s...
1,user,I will give you the response data in chunks of...
2,assistant,Understood. Please provide the first set of 20...
3,user,"[(114782448361, 10, 'What makes GVCA a good ch..."
4,assistant,"```json\n[\n {\n ""count"": 1,\n ""respond..."
...,...,...
138,assistant,"```json\n[\n {\n ""count"": 1,\n ""Respond..."
139,user,"[(114787354873, 10, 'What makes GVCA a good ch..."
140,assistant,"```json\n[\n {\n ""count"": 1,\n ""Respond..."
141,user,"[(114787906460, 10, 'What makes GVCA a good ch..."


In [14]:
# Filter the dataframe for rows with role 'assistant' and content that includes JSON data (identified by ```json)
df_assistant_json = df_responses[(df_responses['role'] == 'assistant') & (df_responses['content'].str.contains("```json", na=False))]

# Display the filtered dataframe
df_assistant_json.head()

,role,content
4,assistant,"```json\n[\n {\n ""count"": 1,\n ""respond..."
6,assistant,"```json\n[\n {\n ""count"": 18,\n ""respon..."
8,assistant,"```json\n[\n {\n ""count"": 33,\n ""respon..."
10,assistant,"```json\n[\n {\n ""count"": 49,\n ""respon..."
12,assistant,"```json\n[\n {\n ""count"": 67,\n ""respon..."


In [15]:
import json

combined_data = []
for content in df_assistant_json['content']:
    # Remove the surrounding code block markers
    json_str = content.strip().lstrip('`').replace('json', '', 1).strip()
    if json_str.endswith('```'):
        json_str = json_str[:-3].strip()
    try:
        # Load the JSON data (a list) and extend the combined list
        data = json.loads(json_str)
        combined_data.extend(data)
    except Exception as e:
        print("Error parsing JSON content:", e)

# Create a new DataFrame from the combined JSON data
df_combined = pd.DataFrame(combined_data)
df_combined.head()

,count,respondent_id,question_id,grade_level,categories,RespondentID,QuestionID,Grade Level
0,1,1.147824e+11,10.0,Grammar,Curriculum,NaN,NaN,NaN
1,2,1.147824e+11,10.0,Whole School,"Culture & Virtues,Curriculum,Teachers",NaN,NaN,NaN
2,3,1.147824e+11,11.0,Grammar,"Communication,Extra-curriculars & Sports,Concern",NaN,NaN,NaN
3,4,1.147824e+11,11.0,Whole School,"Communication,Extra-curriculars & Sports,Concern",NaN,NaN,NaN
4,5,1.147825e+11,10.0,Grammar,"Curriculum,Culture & Virtues",NaN,NaN,NaN


In [16]:
df_combined[df_combined['RespondentID'].notna()]

,count,respondent_id,question_id,grade_level,categories,RespondentID,QuestionID,Grade Level
879,1,NaN,NaN,NaN,Policies & Administration,1.147872e+11,10.0,Whole School
880,2,NaN,NaN,NaN,Other,1.147872e+11,11.0,Grammar
881,3,NaN,NaN,NaN,Other,1.147872e+11,11.0,Whole School
882,4,NaN,NaN,NaN,"Curriculum,Good Outcomes",1.147872e+11,10.0,Grammar
883,5,NaN,NaN,NaN,"Facilities,Policies & Administration",1.147872e+11,11.0,Whole School
...,...,...,...,...,...,...,...,...
967,11,NaN,NaN,NaN,"Policies & Administration, Concern",1.147881e+11,11.0,Whole School
968,12,NaN,NaN,NaN,"Curriculum, Extra-curriculars & Sports",1.147881e+11,10.0,Grammar
969,13,NaN,NaN,NaN,"Culture & Virtues, Good Outcomes",1.147881e+11,10.0,Whole School
970,14,NaN,NaN,NaN,Communication,1.147881e+11,11.0,Grammar


In [17]:
# For rows where 'RespondentID' is not null, update the left-side columns
mask = df_combined['RespondentID'].notna()

df_combined.loc[mask, 'respondent_id'] = df_combined.loc[mask, 'RespondentID']
df_combined.loc[mask, 'question_id'] = df_combined.loc[mask, 'QuestionID']
df_combined.loc[mask, 'grade_level'] = df_combined.loc[mask, 'Grade Level']

# Remove the extra columns
df_combined.drop(columns=['RespondentID', 'QuestionID', 'Grade Level'], inplace=True)

# Display the updated DataFrame
df_combined

,count,respondent_id,question_id,grade_level,categories
0,1,1.147824e+11,10.0,Grammar,Curriculum
1,2,1.147824e+11,10.0,Whole School,"Culture & Virtues,Curriculum,Teachers"
2,3,1.147824e+11,11.0,Grammar,"Communication,Extra-curriculars & Sports,Concern"
3,4,1.147824e+11,11.0,Whole School,"Communication,Extra-curriculars & Sports,Concern"
4,5,1.147825e+11,10.0,Grammar,"Curriculum,Culture & Virtues"
...,...,...,...,...,...
967,11,1.147881e+11,11.0,Whole School,"Policies & Administration, Concern"
968,12,1.147881e+11,10.0,Grammar,"Curriculum, Extra-curriculars & Sports"
969,13,1.147881e+11,10.0,Whole School,"Culture & Virtues, Good Outcomes"
970,14,1.147881e+11,11.0,Grammar,Communication


In [21]:
unique_categories = sorted({
	cat.strip()
	for cats in df_combined['categories']
	for cat in (cats if isinstance(cats, list) else cats.split(','))
})
unique_categories

['Communication',
 'Community',
 'Concern',
 'Culture & Virtues',
 'Curriculum',
 'Extra-curricular & Sports',
 'Extra-curriculars & Sports',
 'Facilities',
 'Good Outcomes',
 'Other',
 'Policies & Administration',
 'Teachers',
 'Wellbeing']

In [22]:
df_combined['categories'] = df_combined['categories'].str.replace('Extra-curricular & Sports', 'Extra-curriculars & Sports')
df_combined.head()

,count,respondent_id,question_id,grade_level,categories
0,1,1.147824e+11,10.0,Grammar,Curriculum
1,2,1.147824e+11,10.0,Whole School,"Culture & Virtues,Curriculum,Teachers"
2,3,1.147824e+11,11.0,Grammar,"Communication,Extra-curriculars & Sports,Concern"
3,4,1.147824e+11,11.0,Whole School,"Communication,Extra-curriculars & Sports,Concern"
4,5,1.147825e+11,10.0,Grammar,"Curriculum,Culture & Virtues"


In [24]:
unique_categories = sorted({
    cat.strip()
    for cats in df_combined['categories'].dropna()
    for cat in cats.split(',')
})
unique_categories

['Communication',
 'Community',
 'Concern',
 'Culture & Virtues',
 'Curriculum',
 'Extra-curriculars & Sports',
 'Facilities',
 'Good Outcomes',
 'Other',
 'Policies & Administration',
 'Teachers',
 'Wellbeing']

In [25]:
print("Unique grade levels:", df_combined['grade_level'].unique())
print("Unique question ids:", df_combined['question_id'].unique())

Unique grade levels: ['Grammar' 'Whole School' 'Middle' 'High' 'What makes GVCA a go' None
 'Whole' 'None']
Unique question ids: [10. 11.]


In [26]:
# Define the list of valid grade levels
valid_grade_levels = ['Grammar', 'Whole School', 'Middle', 'High']

# Filter the DataFrame for rows where the grade_level is not in the valid list
invalid_grade_levels = df_combined[~df_combined['grade_level'].isin(valid_grade_levels)]

# Display the filtered DataFrame
invalid_grade_levels

,count,respondent_id,question_id,grade_level,categories
31,32,1.147825e+11,10.0,What makes GVCA a go,Other
44,45,1.147825e+11,10.0,None,Other
45,46,1.147825e+11,11.0,None,Other
47,48,1.147825e+11,10.0,Whole,Other
65,66,1.147826e+11,11.0,None,Other
...,...,...,...,...,...
923,7,1.147873e+11,11.0,None,Other
938,2,1.147872e+11,11.0,None,Other
941,5,1.147873e+11,11.0,None,Other
943,7,1.147873e+11,11.0,None,Other


In [27]:
# Replace the value in grade_level for the rows in invalid_grade_levels with "Defective Data"
df_combined.loc[invalid_grade_levels.index, 'grade_level'] = "Defective Data"

# Display the updated DataFrame
df_combined

,count,respondent_id,question_id,grade_level,categories
0,1,1.147824e+11,10.0,Grammar,Curriculum
1,2,1.147824e+11,10.0,Whole School,"Culture & Virtues,Curriculum,Teachers"
2,3,1.147824e+11,11.0,Grammar,"Communication,Extra-curriculars & Sports,Concern"
3,4,1.147824e+11,11.0,Whole School,"Communication,Extra-curriculars & Sports,Concern"
4,5,1.147825e+11,10.0,Grammar,"Curriculum,Culture & Virtues"
...,...,...,...,...,...
967,11,1.147881e+11,11.0,Whole School,"Policies & Administration, Concern"
968,12,1.147881e+11,10.0,Grammar,"Curriculum, Extra-curriculars & Sports"
969,13,1.147881e+11,10.0,Whole School,"Culture & Virtues, Good Outcomes"
970,14,1.147881e+11,11.0,Grammar,Communication


In [28]:
unique_grade_levels = df_combined['grade_level'].unique()
print("Unique grade levels:", unique_grade_levels)

Unique grade levels: ['Grammar' 'Whole School' 'Middle' 'High' 'Defective Data']


In [ ]:
df_combined = df_combined.drop_duplicates(subset=['respondent_id', 'question_id', 'grade_level'])
df_combined

In [ ]:
from sqlalchemy import create_engine, text
from utilities import load_env_vars

# Retrieve the environment variables defined globally in this cell.
INPUT_FILEPATH, DATABASE_SCHEMA, DATABASE_CONNECTION_STRING = load_env_vars()

# Use the environmental variables to establish database connection and schema.
engine = create_engine(DATABASE_CONNECTION_STRING)

# Define the CREATE TABLE query using the DATABASE_SCHEMA variable.
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {DATABASE_SCHEMA}.ai_survey_analysis (
    count INTEGER,
    respondent_id BIGINT,
    question_id INTEGER,
    grade_level VARCHAR(50),
    categories TEXT
);
"""

# Execute the query to create the table.
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()

# Load the DataFrame directly into the PostgreSQL table using the schema from the environment.
df_combined.to_sql(
    'ai_survey_analysis',
    engine,
    schema=DATABASE_SCHEMA,
    if_exists='replace',
    index=False
)

972